<a href="https://colab.research.google.com/github/shiehn/rune_notebooks/blob/main/elixir_musicgen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL MUSIC_GEN

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!python -m pip install setuptools wheel
!python -m pip install -U audiocraft
!pip install ffmpeg-python
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121


from audiocraft.models import MusicGen
from audiocraft.utils.notebook import display_audio
from audiocraft.data.audio import audio_write
import torchaudio

model = MusicGen.get_pretrained('medium', device='cuda')


!pip install runes-client


import os
import soundfile as sf
import runes_client.core as runes
from runes_client import RunesFilePath

placeholder_txt = "Enter the token generated by the Crucible plugin"
CONNECTION_TOKEN = "816ebfd1-8be1-4e87-bc2c-d828d0f3078a" #@param {type:"string"}
runes_token = CONNECTION_TOKEN

if runes_token is None or runes_token == "" or runes_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

async def text_two_music_func(audio_description: str = "hiphop",
                      audio_length: int = 8,
                      num_samples: int = 1):
    try:
        model.set_generation_params(duration=audio_length)

        for i in range(num_samples):
          output = model.generate(
              descriptions=[audio_description],
              progress=True, return_tokens=True
          )

          output_audio_name = f"generated_audio_{i}.wav"

          # Move the tensor to CPU, convert to numpy array, reshape and ensure the data type is float32
          generated_audio = output[0].cpu().numpy().astype('float32')
          generated_audio = generated_audio.squeeze()  # Remove extra dimensions
          sf.write(output_audio_name, generated_audio, 32000, subtype='PCM_16')  # Save as WAV file


          # after executing your custom code you send data back to the plugin like so ..
          await runes.output().add_file(output_audio_name)


        await runes.output().add_message("Returning the input file")
        await runes.output().send()

        return True
    except Exception as e:
        print(f"Error in arbitrary_method: {e}")
        return f"Method encountered an error: {e}"


runes.set_token(token=runes_token)
runes.set_name("MusicGen Text-To-Audio")
runes.set_description("Provide a text description, number of samples, and sample lenth in seconds")
runes.register_method(text_two_music_func)

runes.connect_to_server()

See our repo https://github.com/facebookresearch/audiocraft for more details on how to use this model!

See also [MusicGen Gradio Demo](https://colab.research.google.com/drive/1-Xe9NCdIs2sCUbiSmwHXozK6AAhMm7_i?usp=sharing) for a Colab using the Gradio app instead!
